# DNS Resolution and Caching in Python
Networking BSc – Jupyter Notebook Exercise (~45 minutes)

In this exercise you will:
1. Perform basic DNS queries in Python.
2. Compare how different DNS servers respond.
3. Implement a simple DNS cache with TTL handling.

> **Prerequisites:**
> - Basic Python (functions, lists, dictionaries)
> - Basic understanding of DNS (A records, TTL)


## Part 0 – Setup
Run the following cell once to install the required library (`dnspython`).

In [1]:
# This may take a few seconds the first time.
!pip install dnspython

Defaulting to user installation because normal site-packages is not writeable


## Part 1 – Basic DNS Query
In this part you will perform a simple DNS query for an **A record** (IPv4 address).

**Task 1.1** – Run the code below and observe the output. Then change the domain
name to a few other sites (e.g., `www.hit.ac.il`, `www.python.org`) and see how the
results change.


In [2]:
import dns.resolver

domain = "www.google.com"  # TODO: change to other domains and re-run
qtype = "A"  # A record (IPv4)

answer = dns.resolver.resolve(domain, qtype)
print(f"DNS answer for {domain} ({qtype}):")
for rdata in answer:
    print("IP:", rdata.address)

print("TTL:", answer.rrset.ttl)


DNS answer for www.google.com (A):
IP: 142.251.141.4
TTL: 246


**Questions for reflection:**
1. What is an **A record**?
2. What does **TTL** (Time To Live) mean in the context of DNS?
3. Why might different domains have different TTL values?


## Part 2 – Comparing Different DNS Servers
Real systems can use different DNS resolvers (e.g., Google's `8.8.8.8`, Cloudflare's `1.1.1.1`,
or ISP-provided resolvers). In this part you will compare responses from different DNS servers.

**Task 2.1** – Complete and run the function below to query a specific DNS server and measure
the response time.

In [ ]:
import dns.resolver
import time

def query_dns(domain, server_ip, qtype="A"): #כשאני נרשמת לרשת אני מקבלת את הserver_ip
    """Query a specific DNS server for a domain and return IPs, TTL, and latency (ms)."""
    resolver = dns.resolver.Resolver()
    resolver.nameservers = [server_ip]

    start = time.perf_counter()
    answer = resolver.resolve(domain, qtype)
    elapsed_ms = (time.perf_counter() - start) * 1000

    ips = [r.address for r in answer]
    ttl = answer.rrset.ttl
    return ips, ttl, elapsed_ms


**Task 2.2** – Use the function to query several domains and compare the results between
two or more DNS servers.

You can start with Google DNS (`8.8.8.8`) and Cloudflare DNS (`1.1.1.1`).

In [ ]:
domains = [
    "www.google.com",
    "www.hit.ac.il",
    "dns.cs.umass.edu",  # can be changed to any domain
    #להוסיף dns לוקאלי שלי 
]

servers = {
    "Google DNS": "8.8.8.8",
    "Cloudflare DNS": "1.1.1.1",
    # TODO: optionally add another DNS server (e.g., your ISP resolver)
}

for d in domains:
    print(f"\nDomain: {d}")
    for name, ip in servers.items():
        try:
            ips, ttl, t = query_dns(d, ip)
            print(f"  Server: {name:12} | IPs: {ips} | TTL: {ttl} | Time: {t:.2f} ms")
        except Exception as e:
            print(f"  Server: {name:12} | ERROR: {e}")



Domain: www.google.com
  Server: Google DNS   | IPs: ['172.217.168.196'] | TTL: 300 | Time: 228.73 ms
  Server: Cloudflare DNS | IPs: ['142.250.187.100'] | TTL: 300 | Time: 102.51 ms

Domain: www.hit.ac.il
  Server: Google DNS   | IPs: ['192.114.5.142'] | TTL: 21600 | Time: 195.86 ms
  Server: Cloudflare DNS | IPs: ['192.114.5.142'] | TTL: 43200 | Time: 158.06 ms

Domain: dns.cs.umass.edu
  Server: Google DNS   | ERROR: The DNS query name does not exist: dns.cs.umass.edu.
  Server: Cloudflare DNS | ERROR: The DNS query name does not exist: dns.cs.umass.edu.


**Questions for reflection:**
1. Do all DNS servers return the same IP addresses for each domain?
2. Are the TTL values always the same? Why might they differ?
3. Which DNS server seems faster in your tests? What factors could influence this?


## Part 3 – Implementing a Simple DNS Cache
DNS resolvers cache responses for a period defined by the TTL to avoid repeated network queries.
In this part you will implement a **very simple DNS cache** in Python.

We will store cached entries in a dictionary with this structure:

- **key**: `(domain, qtype)`
- **value**: `(ips_list, expiry_time)` – where `expiry_time = now + ttl`


In [5]:
import dns.resolver
import time

# Global in-memory cache
cache = {}
# key: (domain, qtype)
# value: (ips_list, expiry_time)

def resolve_with_cache(domain, qtype="A", server_ip="8.8.8.8"):
    """Resolve a domain using a simple in-memory cache with TTL support."""
    now = time.time()
    key = (domain, qtype)

    # 1. Check if in cache and not expired
    if key in cache:
        ips, expiry = cache[key]
        if now < expiry:
            print(f"[CACHE HIT] {domain} -> {ips}")
            return ips
        else:
            print(f"[CACHE EXPIRED] {domain}")
            del cache[key]

    # 2. Not in cache or expired → query DNS server
    resolver = dns.resolver.Resolver()
    resolver.nameservers = [server_ip]
    answer = resolver.resolve(domain, qtype)
    ips = [r.address for r in answer]
    ttl = answer.rrset.ttl

    # 3. Store in cache with expiry time
    expiry_time = now + ttl
    cache[key] = (ips, expiry_time)
    print(f"[NETWORK QUERY] {domain} -> {ips}, TTL={ttl}")
    return ips


**Task 3.1** – Run the following cell several times and observe when the cache is used
and when a network query is performed.

Try changing the `sleep` duration or domain name and see the effect.

In [6]:
test_domain = "www.google.com"  # You can change this

for i in range(3):
    print(f"\nQuery #{i+1}")
    resolve_with_cache(test_domain)
    time.sleep(2)  # wait 2 seconds between queries



Query #1
[NETWORK QUERY] www.google.com -> ['172.217.168.196'], TTL=300

Query #2
[CACHE HIT] www.google.com -> ['172.217.168.196']

Query #3
[CACHE HIT] www.google.com -> ['172.217.168.196']


**Task 3.2 (Optional, for exploration)**
- Print the `cache` dictionary to see what is stored.
- Manually simulate a short TTL by overriding `ttl` in the code (e.g., `ttl = 5`) and see
  how quickly entries expire.
- Extend the cache to count how many times each domain was served from cache vs. from the network.


## Summary
In this exercise you:
- Sent DNS queries from Python using `dnspython`.
- Compared answers and performance from different DNS servers.
- Implemented a basic DNS cache with TTL-based expiration.

These ideas generalize to real recursive resolvers, OS-level DNS caching, and performance
optimizations in large-scale systems.
